In [2]:
import cv2 as cv
import matplotlib.pyplot as plt
import torch
import torchvision
import os

In [6]:
png_count = 0
jpg_count = 0
jpeg_count = 0

# Loop through the files in the folder
for filename in os.listdir("../Data/images"):
    if filename.endswith(".png"):
        png_count += 1
    elif filename.endswith((".jpg")):
        jpg_count += 1
    elif filename.endswith((".jpeg")):
        jpeg_count += 1
    else:
        print(filename)

print(png_count, jpg_count, jpeg_count)

16745_2_1.PNG
16745_2_2.PNG
16745_2_3.PNG
16745_7_5.PNG
4c8dee3d9fe81567b98ed1b0b2b4c6_jumbo.JPG
covid-19-pneumonia-mild.JPG
205 510 209
